In [1]:
import numpy as np
from numberline import *
import voting_rules as rules

In [2]:
def get_continuous_weights(values: [float], low: float, high: float) -> [tuple]:
    """
    Returns a list of value-weight pairs for the given values assuming the
    population is infinite, as per "Proxy Voting for Better Outcomes"
    chapter 2, paragraph 2.

    # Figure 1 second line of paper
    >>> v = [1, 3, 6, 7, ]
    >>> get_continuous_weights(v, 0, 10)
    [(1, 2.0), (3, 2.5), (6, 2.0), (7, 3.5)]

    # Figure 1 third line of paper
    >>> v = [1, 6, 7, ]
    >>> get_continuous_weights(v, 0, 10)
    [(1, 3.5), (6, 3.0), (7, 3.5)]
    """
    if len(values) == 0:
        return []

    low, high = min(low, high), max(low, high)

    values = sorted(values)
    assert low <= values[0] <= high, "Values must be in the range [min_val, max_val]"
    assert low <= values[-1] <= high, "Values must be in the range [min_val, max_val]"

    weights = [None] * len(values)

    # prev_val_border will keep track of where the last value's range ends
    prev_val_border = low
    # Loop through all but the last, calculating the weights
    for i in range(len(values) - 1):
        val = values[i]
        next_val = values[i + 1]
        weight = val - prev_val_border + (next_val - val) / 2
        weights[i] = weight
        # The new border will be halfway between the current value and the next
        prev_val_border = val + (next_val - val) / 2

    weights[-1] = high - prev_val_border
    return list(zip(values, weights))

In [ ]:
min_val = 0
max_val = 10
size = 4

true_pref = (max_val - min_val) / 2 + min_val
vals = np.random.uniform(low=min_val, high=max_val, size=size)
print(vals)

In [ ]:
ax = numberline(vals, min_val, max_val)

unweighted_result = rules.mean(vals)
unweighted_distance = abs(unweighted_result - true_pref)

weighted_vals = get_continuous_weights(vals, min_val, max_val)
weighted_result = rules.weighted_mean(weighted_vals)
weighted_distance = abs(weighted_result - true_pref)

ax.scatter(true_pref, 0, clip_on=False, zorder=3, color='green')
ax.scatter(unweighted_result, 0, clip_on=False, zorder=3, color='red')
ax.scatter(weighted_result, 0, clip_on=False, zorder=3, color='purple')

print(f"True preference (green): {true_pref}")
print(f"Unweighted result (red): {unweighted_result}")
print(f"\tDistance: {unweighted_distance}")
print(f"Weighted result (purple): {weighted_result}")
print(f"\tDistance: {weighted_distance}")
print(f"\nBetter result: {'UNWEIGHTED' if unweighted_distance <= weighted_distance else 'WEIGHTED'}")
print(f"\tDifference: {abs(unweighted_distance - weighted_distance)}")

# Testing

In [5]:
should_test = False
if should_test:
    import doctest
    doctest.testmod()